<a href="https://colab.research.google.com/github/galnov/hello-world/blob/main/Draw_Me_a_Sheep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we'll generate an image using stability.ai's API.

As a first step, please go to https://beta.dreamstudio.ai/account and copy your API key.

Paste it as a secret in Google Colab with key name "stability-ai-key".

Enable notebook access to key.

In [1]:
import requests
import json
import base64
from google.colab import userdata

api_addr = 'https://api.stability.ai'
api_key = userdata.get('stability-ai-key')

Code in the next cell will get the list of currently available stability.ai engines.

In [2]:
def getEnginesList():
    url = f"{api_addr}/v1/engines/list"
    response = requests.get(url, headers={"Authorization": f"Bearer {api_key}"})

    if response.status_code == 200:
      engines = json.loads(response.text)
      #pretty_json = json.dumps(response.json(), indent=4)
      #print(pretty_json)
      return engines
    else:
      print("Error getting engines list: {}".format(response.status_code))

engines = getEnginesList()

Using the last engine, stability.ai will create 3 images based on the given prompt. Images can be accessed through Google Colab's Files section.

In [3]:
engine_id = engines[-1]['id']
#print("Engine ID: {}".format(engine_id))

prompt = '''Create a photorealistic high resolution image of an oil painting
in which a robot artist stands in a room and draws an oil painting of itself
drawing an image recursively. The painting will be drawn on canvas placed
on a wooden stand.'''
height = 512
width = 512
steps = 50

def generateStableDiffusionImage(prompt, height, width, steps):
    url = f"{api_addr}/v1/generation/{engine_id}/text-to-image"
    headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
    }
    payload = {}
    payload['text_prompts'] = [{"text": f"{prompt}"}]
    payload['cfg_scale'] = 7
    payload['clip_guidance_preset'] = 'FAST_BLUE'
    payload['height'] = height
    payload['width'] = width
    payload['samples'] = 3
    payload['steps'] = steps

    response = requests.post(url,headers=headers,json=payload)

    #Processing the response
    if response.status_code == 200:
        data = response.json()
        for i, image in enumerate(data["artifacts"]):
            with open(f"v1_txt2img_{i}.png", "wb") as f:
                f.write(base64.b64decode(image["base64"]))

generateStableDiffusionImage(prompt, height, width, steps)